# About: Azure仮想マシンの作成

---

Moodle環境を構築するためのAzure仮想マシンを作成します。

## 概要

### 前提条件

このNotebookでAzureに仮想マシンを作成する際の前提条件を以下に示します。

* Azureの[サービスプリンシパル](https://docs.microsoft.com/ja-jp/cli/azure/create-an-azure-service-principal-azure-cli?view=azure-cli-latest)

* 起動したAzureの仮想マシンに対して、このNotebook環境からsshでログイン可能であること
    - ネットワーク的に到達可能なこと
    - ログインするためのSSH鍵ファイルをこのNotebook環境に配置してあること

### 準備

Azureの仮想マシンを作成するために必要となるライブラリをインストールします。

現在の環境にインストールされているライブラリとの競合をさけるためにpipenvで独立した環境を構築します。

まず pipenv をインストールします。

In [ ]:
!pip install --user pipenv

pipenvの環境に ansible で Azureを操作するのに必要となるライブラリをインストールします。

In [ ]:
import os
os.environ['PATH'] += f':{os.environ["HOME"]}/.local/bin'
!pipenv install ansible
!pipenv run ansible-galaxy collection install azure.azcollection
!pipenv install -r $HOME/.ansible/collections/ansible_collections/azure/azcollection/requirements-azure.txt

## パラメータの設定

このNotebookで作成するAzure仮想マシンのパラメータを設定します。

### Azureの認証情報の設定

Azureの操作を行うために必要とサービスプリンシパルに関する情報を指定します。

> この節で指定するAzureのサービスプリンシパルを取得する手順については「[Azure CLI で Azure サービス プリンシパルを作成する](https://docs.microsoft.com/ja-jp/cli/azure/create-an-azure-service-principal-azure-cli?view=azure-cli-latest)」などを参照してください。

次のセルを実行すると入力枠が表示されるのでサブスクリプションIDの値を入力してください。

> 入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
azure_subscription_id = getpass()

テナントIDを入力してください。

In [ ]:
azure_tenant = getpass()

クライアントIDを入力してください。

In [ ]:
azure_client_id = getpass()

シークレットを入力してください。

In [ ]:
azure_secret = getpass()

このNotebookから実行するコマンドでAzureのサービスプリンシパルを参照できるようにするために、[環境変数(`AZURE_SUBSCRIPTION_ID`, `AZURE_TENANT`, `AZURE_CLIENT_ID`, `AZURE_SECRET`)](https://docs.microsoft.com/ja-jp/azure/developer/ansible/install-on-linux-vm#env-credentials)の設定を行います。

In [ ]:
import os

os.environ['AZURE_SUBSCRIPTION_ID'] = azure_subscription_id
os.environ['AZURE_TENANT'] = azure_tenant
os.environ['AZURE_CLIENT_ID'] = azure_client_id
os.environ['AZURE_SECRET'] = azure_secret

### 仮想マシンの名前の指定

In [ ]:
# (例)
# azure_vm_name = 'moodle'

azure_vm_name =

### リソースグループの指定

仮想マシンを作成するリソースグループを指定してください。

In [ ]:
# (例)
# azure_resource_group = 'resource-group'

azure_resource_group =

### ネットワークの指定

仮想ネットワーク名を指定してください。

In [ ]:
# (例)
# azure_virtual_network_name = 'virtual-network-name'

azure_virtual_network_name =

サブネット名を指定してください。

In [ ]:
# (例)
# azure_subnet_name = 'subnet-name'

azure_subnet_name =

### 仮想マシンに割り当てるリソースの指定

起動する仮想マシンに割り当てるリソース量を指定します。

仮想マシンの[サイズ](https://docs.microsoft.com/ja-jp/azure/virtual-machines/linux/sizes)を指定してください。

In [ ]:
# (例)
# azure_vm_size = 'Standard_D2_v3'
# azure_vm_size = 'Standard_B2ms'

azure_vm_size =

OSディスクのサイズ(GB)を指定してください。

In [ ]:
# (例)
# azure_os_disk_size_gb = 32

azure_os_disk_size_gb =

### アカウント名の指定

仮想マシンに作成するアカウント名を指定してください。

In [ ]:
# (例)
# azure_admin_user = 'moodle'

azure_admin_user =

### SSH公開鍵の指定

作成した仮想マシンにSSHでログインする際に利用するSSHの公開鍵ファイルのパスを指定してください。

In [ ]:
# (例)
# azure_ssh_public_key = '~/.ssh/id_rsa.pub'

azure_ssh_public_key =

## 仮想マシンの作成

これまでに入力したパラメータに従いAzureの仮想マシンを作成します。

### Ansible Playbook の生成

このNotebookでは仮想マシンを作成するために [Ansible](https://www.ansible.com/)を利用します。

次のセルを実行するとAzureの仮想マシンを作成するための [Ansible Playbook](https://docs.ansible.com/ansible/latest/user_guide/playbooks_intro.html) を生成します。

In [ ]:
%run scripts/edit_conf.py
import json

playbook = create_conf_file('localhost', 'azure.yml')
with playbook.open(mode='w') as f:
    f.write(f'''
- hosts: localhost
  tasks:
  - azure.azcollection.azure_rm_virtualmachine:
      name: {azure_vm_name}
      resource_group: {azure_resource_group}
      image:
        offer: CentOS
        publisher: OpenLogic
        sku: '7.5'
        version: latest
      vm_size: {azure_vm_size}
      os_disk_size_gb: {azure_os_disk_size_gb}
      virtual_network_name: {azure_virtual_network_name}
      subnet_name: {azure_subnet_name}
      admin_username: {azure_admin_user}
      ssh_password_enabled: false
      ssh_public_keys:
      - path: /home/{azure_admin_user}/.ssh/authorized_keys
        key_data: "{{{{lookup('file', '{os.path.expanduser(azure_ssh_public_key)}')}}}}"
''')
generate_edit_link(playbook)

上のリンクをクリックするとブラウザの新しいウィンドウ（タブ）で playbook の編集画面が開きます。編集を行った場合は `ctrl-s` またはメニューの[File]-[Save]で編集結果を保存してください。

Azureの仮想マシンを作成するためのパラメータの詳細についてはAnsibleのドキュメント[「azure.azcollection.azure_rm_virtualmachineモジュール」](https://docs.ansible.com/ansible/latest/collections/azure/azcollection/azure_rm_virtualmachine_module.html)を参照してください。

確認のためplaybookの内容を表示します。

In [ ]:
!cat {playbook}

### 仮想マシンの起動

playbookを実行して仮想マシンを起動します。

In [ ]:
!pipenv run ansible-playbook -c local {playbook}

作成した仮想マシンの情報を取得します。

In [ ]:
out = !pipenv run ansible localhost -c local -m azure.azcollection.azure_rm_virtualmachine_info \
    -a 'resource_group={azure_resource_group} name={azure_vm_name}'
idx = [i for i, x in enumerate(out) if x.startswith('localhost |')][0]
if out[idx] == 'localhost | SUCCESS => {':
    data = json.loads(' '.join(['{'] + out[(idx + 1):]))
    network_interface_names = sum([x['network_interface_names'] for x in data['vms']], [])
    for line in out[idx:]:
        print(line)
else:
    for line in out:
        print(line)
    raise RuntimeError("error!")

仮想マシンに割り当てられたパブリックIPアドレスの値を確認します。

In [ ]:
for netif in network_interface_names:
    out = !pipenv run ansible localhost -c local -m azure.azcollection.azure_rm_publicipaddress_info \
        -a 'resource_group={azure_resource_group} name={netif}'
    idx = [i for i, x in enumerate(out) if x.startswith('localhost |')][0]
    if out[idx] == 'localhost | SUCCESS => {':
        data = json.loads(' '.join(['{'] + out[(idx + 1):]))
        for ipaddr in [x['ip_address'] for x in data['publicipaddresses']]:
            print(ipaddr)
    else:
        for line in out:
            print(line)
        raise RuntimeError("error!")

仮想マシンに割り当てられたプライベートIPアドレスの値を確認します。

In [ ]:
for netif in network_interface_names:
    out = !pipenv run ansible localhost -c local -m azure.azcollection.azure_rm_networkinterface_info \
        -a 'resource_group={azure_resource_group} name={netif}'
    idx = [i for i, x in enumerate(out) if x.startswith('localhost |')][0]
    if out[idx] == 'localhost | SUCCESS => {':
        data = json.loads(' '.join(['{'] + out[(idx + 1):]))
        addrs = [x['private_ip_address']
                 for x in sum([x['ip_configurations']
                               for x in data['networkinterfaces']], [])]
        for ipaddr in addrs:
            print(ipaddr)
    else:
        for line in out:
            print(line)
        raise RuntimeError("error!")

## Ansibleの設定

起動した仮想マシンをAnsibleで操作するための設定を行います。

### パラメータの設定

仮想マシンを登録するAnsibleのグループ名を指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'
# target_group = azure_vm_name  # 仮想マシンのNameと同じグループ名にする場合

target_group = azure_vm_name

このNotebook環境から仮想マシンに接続する際のホスト名（IPアドレス）を指定してください。

In [ ]:
# (例)
# target_hostname = 'moodle.example.org'
# target_hostname = '172.30.1.10'

target_hostname =

仮想マシンにログインするためのSSH秘密鍵のファイルを指定してください。

In [ ]:
# (例)
# ssh_private_key_file = '~/.ssh/id_rsa'

ssh_private_key_file =

### インベントリの作成

Ansibleのインベントリ`inventory.yml`をカレントディレクトリに作成します。

In [ ]:
%run scripts/group.py

inventory = {'all': {'children': {
    target_group: {
        'hosts': {
            target_hostname: {},
        },
        'vars': {
            'ansible_user': azure_admin_user,
            'ansible_ssh_private_key_file': os.path.expanduser(ssh_private_key_file),
            'ansible_python_interpreter': '/usr/bin/python',
        }
    }
}}}

generate_edit_link(update_inventory_yml(inventory))

次のセルを実行すると作成したインベントリの内容を表示します。インベントリの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat inventory.yml

### ansible.cfg の作成

先程、カレントディレクトリに作成した`inventory.yml`をAnsibleのインベントリとして指定するための設定を行います。

> カレントディレクトリにコンフィギュレーションファイル`ansible.cfg`を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
cfg = setup_ansible_cfg()
generate_edit_link(cfg)

次のセルを実行すると作成したコンフィギュレーションファイルの内容を表示します。コンフィギュレーションファイルの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat ansible.cfg

### 仮想マシンへの接続確認

作成した仮想マシンに対して Ansible で接続できることを確認します。

In [ ]:
!ssh-keygen -R {target_hostname} || true
!mkdir -p -m 0700 ~/.ansible/cp
!env ANSIBLE_HOST_KEY_CHECKING=False \
    ansible {target_group} -m ping

正常に接続できると以下のように表示されます。

```
XXX.XXX.XXX.XXX | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
```

`~/.ssh/known_hosts`の内容を更新します。

In [ ]:
!ssh-keyscan -H {target_hostname} >> ~/.ssh/known_hosts

VCノードに対して設定ファイルの変更やパッケージの追加を行う場合にVCノードの管理者権限が必要になる場合があります。Ansibleで管理者権限によるコマンド実行が可能かどうかを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {target_group} -b -a 'whoami'

### group_vars ファイルの更新

In [ ]:
%run scripts/group.py

update_group_vars(
    target_group,
    azure_resource_group=azure_resource_group,
    azure_vm_name=azure_vm_name,
)

## パッケージなどのインストール

Moodle環境を構築する際に必要となるパッケージなどのインストールを行います。

### Dockerのインストール

[Install Docker Engine on CentOS](https://docs.docker.com/engine/install/centos/) の手順に従い Docker のインストールを行います。

まず、必要なパッケージをインストールします。

In [ ]:
!ansible {target_group} -b -m yum -a 'name=yum-utils'

Dockerのレポジトリを追加します。

In [ ]:
!ansible {target_group} -b -a \
    'yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo'

Dockerのパッケージをインストールします。

In [ ]:
!ansible {target_group} -b -m yum -a 'name=docker-ce,docker-ce-cli,containerd.io'

Docker Engine を実行するサービスを開始します。

In [ ]:
!ansible {target_group} -b -a 'systemctl start docker'
!ansible {target_group} -b -a 'systemctl enable docker'

Docker Engine が実行されていることを確認するために `docker info` を実行してみます。

In [ ]:
!ansible {target_group} -b -a 'docker info'

dockerコマンドを管理者権限なしで実行できるようにするためにユーザを `docker` グループに所属させるようにします。

In [ ]:
!ansible {target_group} -b -m user -a 'name={{{{ansible_user}}}} append=yes groups=docker'

管理者権限なしで docker コマンドが実行できることを確認します。まず、新しいグループでログインし直すために、現在のsshの接続をいったん終了します。

In [ ]:
if os.path.exists(os.path.expanduser(f'~/.ansible/cp/{azure_admin_user}@{target_hostname}:22')):
    !ssh -o ControlPath=~/.ansible/cp/{azure_admin_user}@{target_hostname}:22 -O exit {target_hostname}

一般ユーザで `docker info`が実行できることを確認します。

In [ ]:
!ansible {target_group} -a 'docker info'

### docker-compose のインストール

[Install Compose on Linux systems](https://docs.docker.com/compose/install/#install-compose-on-linux-systems)の手順に従い `docker-compose` コマンドをインストールします。

In [ ]:
docker_compose_url = 'https://github.com/docker/compose/releases/download/1.26.2/docker-compose-Linux-x86_64'

In [ ]:
!ansible {target_group} -b -m get_url -a 'url={docker_compose_url} dest=/usr/bin/docker-compose mode=0755'

In [ ]:
!ansible {target_group} -a 'docker-compose version'

### Python3などのインストール

In [ ]:
!ansible {target_group} -b -m yum -a 'name=python3,python3-pip,libselinux-python,libselinux-python3,tree'

### aws cli のインストール

構築したMoodle環境をAmazon S3にバックアップする際に利用する aws cli のインストールを行います。

In [ ]:
!ansible {target_group} -b -e ansible_python_interpreter=/usr/bin/python3 -m pip -a 'name=awscli'

### タイムゾーンの変更

タイムゾーンをJSTに変更します。

In [ ]:
!ansible {target_group} -b -a 'timedatectl set-timezone Asia/Tokyo'

タイムゾーンが変更されたことを確認します。

In [ ]:
!ansible {target_group} -a 'date'